In [ ]:
import pandas as pd
import polars as pl
import numpy  as np
import networkx as nx
from math import sin, cos, pi, sqrt, atan2
from os.path import exists
import time
from rtsvg import *
rt = RACETrack()
df = pl.concat([pl.read_csv('../../data/2013_vast_challenge/mc3_netflow/nf/nf-chunk1.csv'),
                pl.read_csv('../../data/2013_vast_challenge/mc3_netflow/nf/nf-chunk2.csv'),
                pl.read_csv('../../data/2013_vast_challenge/mc3_netflow/nf/nf-chunk3.csv')])
ts1 = time.time()
df = rt.columnsAreTimestamps(df, 'parsedDate')
ts2 = time.time()
print(f'Loading Time ... {ts2 - ts1:0.2} sec')

df = df.drop(['TimeSeconds',
              #'parsedDate',
              'dateTimeStr',
              #'ipLayerProtocol',
              'ipLayerProtocolCode',
              #'firstSeenSrcIp',
              #'firstSeenDestIp',
              #'firstSeenSrcPort',
              #'firstSeenDestPort',
              'moreFragments',
              'contFragments',
              #'durationSeconds',
              'firstSeenSrcPayloadBytes',
              'firstSeenDestPayloadBytes',
              #'firstSeenSrcTotalBytes',
              #'firstSeenDestTotalBytes',
              #'firstSeenSrcPacketCount',
              #'firstSeenDestPacketCount',
              'recordForceOut'])

df = df.rename({'parsedDate':'ts',
               'ipLayerProtocol':'pro',
               'firstSeenSrcIp':'sip',
               'firstSeenDestIp':'dip',
               'firstSeenSrcPort':'spt',
               'firstSeenDestPort':'dpt',
               'durationSeconds':'dur',
               'firstSeenSrcTotalBytes':'soct',
               'firstSeenDestTotalBytes':'doct',
               'firstSeenSrcPacketCount':'spkt',
               'firstSeenDestPacketCount':'dpkt'})

df.sample(3)

In [ ]:
layout_file = '../../data/2013_vast_challenge/mc3_netflow/spring_layout.csv'
relates = [('sip','dip')]
g       = rt.createNetworkXGraph(df, relates)
pos = {} if exists(layout_file) else nx.spring_layout(g)
#rt.link(df, relates, pos)

In [ ]:
df_uniqs = df.unique(subset=['sip','dip'])
total_nodes = len(set(df_uniqs['sip']) | set(df_uniqs['dip']))
print(f'{len(df_uniqs)=} | {total_nodes=}')
_igl_ = rt.interactiveGraphLayout(df_uniqs, {'relationships':relates, 'pos':pos, 'draw_labels':False, 'bounds_percent':0.02}, w=1200, h=800)
if exists(layout_file): _igl_.loadLayout(layout_file)
_igl_

In [ ]:
_igl_.saveLayout(layout_file)

In [ ]:
print(_igl_)